In [1]:
import nltk
nltk.download('stopwords')

import numpy as np
import json
import glob

import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

import spacy
from nltk.corpus import stopwords

import pyLDAvis
import pyLDAvis.gensim

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
c:\Users\Oshadha Abeyrathne\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:9: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
def load_data(file):
    with open (file, "r", encoding="utf-8") as f:
        data = json.load(f) 
    return (data)

def write_data(file, data):
    with open (file, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=4)

In [3]:
stopwords = stopwords.words("english")

In [4]:
print(stopwords)

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [5]:
data = load_data("data2.json")["abstract"]

print (data[0:100])

To effectively monitor the spending, we offer an advanced expense tracker. Everyone who plans to kno


In [6]:
def lemmatization(texts, allowed_postags=["NOUN", "ADJ"]):
    nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
    texts_out = []
    
    doc = nlp(texts)
    new_text = []
    for token in doc:
        if token.pos_ in allowed_postags:
            new_text.append(token.lemma_)
    final = " ".join(new_text)
    texts_out.append(final)
    return (texts_out)


lemmatized_texts = lemmatization(data)
print (lemmatized_texts[0])

spending advanced expense tracker expense approach receipt user application user own category food clothe rent bill upper low bound category project specific user demographic age range available track spending overall goal project user overall spending behavior mobile application user transaction receipt


In [7]:
def gen_words(texts):
    final = []
    for text in texts:
        new = gensim.utils.simple_preprocess(text, deacc=True)
        final.append(new)
    return (final)

data_words = gen_words(lemmatized_texts)

print (data_words[0])

['spending', 'advanced', 'expense', 'tracker', 'expense', 'approach', 'receipt', 'user', 'application', 'user', 'own', 'category', 'food', 'clothe', 'rent', 'bill', 'upper', 'low', 'bound', 'category', 'project', 'specific', 'user', 'demographic', 'age', 'range', 'available', 'track', 'spending', 'overall', 'goal', 'project', 'user', 'overall', 'spending', 'behavior', 'mobile', 'application', 'user', 'transaction', 'receipt']


In [8]:
id2word = corpora.Dictionary(data_words)

corpus = []
for text in data_words:
    new = id2word.doc2bow(text)
    corpus.append(new)

print (corpus[0][0:20])

word = id2word[[0][:1][0]]
print (word)

[(0, 1), (1, 1), (2, 2), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 2), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 1), (15, 1), (16, 2), (17, 1), (18, 2), (19, 1)]
advanced


In [9]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5,
                                           random_state=100,
                                           update_every=1,
                                           passes=10)

In [10]:
lda_model.print_topics()

[(0,
  '0.034*"user" + 0.034*"receipt" + 0.034*"project" + 0.034*"category" + 0.034*"goal" + 0.034*"application" + 0.034*"mobile" + 0.034*"age" + 0.034*"spending" + 0.034*"behavior"'),
 (1,
  '0.035*"user" + 0.035*"expense" + 0.035*"project" + 0.035*"spending" + 0.035*"category" + 0.035*"overall" + 0.035*"application" + 0.035*"receipt" + 0.034*"rent" + 0.034*"age"'),
 (2,
  '0.034*"user" + 0.034*"receipt" + 0.034*"project" + 0.034*"mobile" + 0.034*"spending" + 0.034*"low" + 0.034*"age" + 0.034*"goal" + 0.034*"bill" + 0.034*"tracker"'),
 (3,
  '0.034*"user" + 0.034*"category" + 0.034*"project" + 0.034*"expense" + 0.034*"age" + 0.034*"spending" + 0.034*"behavior" + 0.034*"receipt" + 0.034*"rent" + 0.034*"food"'),
 (4,
  '0.111*"user" + 0.068*"spending" + 0.047*"receipt" + 0.047*"application" + 0.047*"overall" + 0.047*"category" + 0.047*"project" + 0.047*"expense" + 0.026*"clothe" + 0.026*"transaction"')]

In [11]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word, mds="mmds", R=30)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.018142  0.003382       1        1  98.093245
1     -0.004455 -0.000824       2        1   0.476705
3     -0.004563 -0.000851       3        1   0.476686
0     -0.004565 -0.000836       4        1   0.476683
2     -0.004559 -0.000871       5        1   0.476682, topic_info=           Term      Freq     Total Category  logprob  loglift
28         user  4.000000  4.000000  Default  29.0000  29.0000
23     spending  2.000000  2.000000  Default  28.0000  28.0000
11      expense  1.000000  1.000000  Default  27.0000  27.0000
18      project  1.000000  1.000000  Default  26.0000  26.0000
8      category  1.000000  1.000000  Default  25.0000  25.0000
..          ...       ...       ...      ...      ...      ...
16      overall  0.006739  1.917202   Topic5  -3.3673  -0.3046
2   application  0.006739  1.917485   Topic5  -3.3673  -0.3047
20      receipt  0.006739  1.917680   Topic5  -3.3673  -0.3048
23     spending  0.006739  2.777305   Topic5  -3.3673  -0.6752
28         user  0.006739  4.497144   Topic5  -3.3673  -1.1572

[174 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
0         1  0.944696     advanced
1         1  0.945320          age
2         1  1.043033  application
3         1  0.945033     approach
4         1  0.944792    available
5         1  0.945279     behavior
6         1  0.944927         bill
7         1  0.945107        bound
8         1  1.043313     category
9         1  0.944415       clothe
10        1  0.945093  demographic
11        1  1.043782      expense
12        1  0.944781         food
13        1  0.945250         goal
14        1  0.944907          low
15        1  0.944893       mobile
16        1  1.043187      overall
17        1  0.945101          own
18        1  1.043569      project
19        1  0.944952        range
20        1  1.042927      receipt
21        1  0.945328         rent
22        1  0.944781     specific
23        1  1.080184     spending
24        1  0.944787        track
25        1  0.944760      tracker
26        1  0.944651  transaction
27        1  0.944915        upper
28        1  0.889453         user, R=29, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 4, 1, 3])